In [ ]:
from google.colab import files
uploaded = files.upload()


Saving BlueSky_Final_Depressed.csv to BlueSky_Final_Depressed.csv


In [ ]:
!pip install requests pandas

In [ ]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=6ca976a171fce5950b76d4f157dcf1cd65ecb406a225b88b67c732de7f8c9253
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
pip install python-dotenv

In [ ]:
python-dotenv

In [1]:
import langdetect
import pandas as pd
import requests
import time
from dotenv import load_dotenv


load_dotenv()

# 🔧 Configure
USERNAME = os.getenv("BLUESKY_USERNAME")
APP_PASSWORD = os.getenv("BLUESKY_PASSWORD")


#Change the csv file name accordingly
INPUT_CSV = "BlueSky__Final_Random_Users.csv"
OUTPUT_CSV = "BlueSky_Random_Users_Previous_Posts.csv"
POSTS_PER_USER = 1000  # Max number of posts to fetch per user

def get_auth_token():
    response = requests.post(
        "https://bsky.social/xrpc/com.atproto.server.createSession",
        json={"identifier": USERNAME, "password": APP_PASSWORD}
    )
    response.raise_for_status()
    return response.json()["accessJwt"]


def fetch_user_posts(handle, token, limit=POSTS_PER_USER):
    posts = []
    cursor = None
    fetched = 0

    headers = {"Authorization": f"Bearer {token}"}

    while fetched < limit:
        params = {
            "actor": handle,
            "limit": min(100, limit - fetched)
        }
        if cursor:
            params["cursor"] = cursor

        res = requests.get(
            "https://bsky.social/xrpc/app.bsky.feed.getAuthorFeed",
            headers=headers,
            params=params
        )

        if res.status_code != 200:
            print(f"❌ Failed to fetch for {handle}: {res.status_code}")
            break

        data = res.json()
        feed = data.get("feed", [])
        if not feed:
            break

        for item in feed:
            post = item.get("post", {}).get("record", {})
            text = post.get("text", "")

            # Filter: 50+ characters and English
            if len(text) >= 50:
                try:
                    lang = langdetect.detect(text)
                    if lang != "en":
                        continue
                except:
                    continue

                posts.append({
                    "username": handle,
                    "text": text,
                    "created_at": post.get("createdAt", ""),
                    "uri": item.get("post", {}).get("uri", ""),
                    "cid": item.get("post", {}).get("cid", "")
                })

        fetched += len(feed)
        cursor = data.get("cursor")
        if not cursor:
            break
        time.sleep(0.5)

    return posts


# Read uploaded CSV
df = pd.read_csv(INPUT_CSV)
usernames = df["username"].dropna().unique()

print(f"🔍 Fetching posts for {len(usernames)} users...")
token = get_auth_token()
all_posts = []

for i, user in enumerate(usernames, 1):
    print(f"({i}/{len(usernames)}) Fetching posts for {user}...")
    try:
        posts = fetch_user_posts(user, token)
        all_posts.extend(posts)
    except Exception as e:
        print(f"⚠️ Error for {user}: {e}")
        continue

print(f"💾 Saving {len(all_posts)} posts to {OUTPUT_CSV}...")
pd.DataFrame(all_posts).to_csv(OUTPUT_CSV, index=False)
print("✅ Done.")


ModuleNotFoundError: No module named 'langdetect'